In [1]:
import os
from copy import deepcopy

import openmm as mm
from openmm import unit
from openmm import app
from pdbfixer import PDBFixer
import mdtraj as md
import nglview as nv
import requests
import numpy as np
from math import sqrt
from tqdm import tqdm
import matplotlib.pyplot as plt

# Playing with the potential energy

## Building the Barnase-Barstar complex

In [2]:
# Function to download PDB files

def fetch_pdb(pdb_id, download_path="./"):

        url = 'http://files.rcsb.org/download/{}.pdb'.format(pdb_id)
        try:
            res = requests.get(url, allow_redirects=True)
        except:
            print("Could not fetch pdb from {}".format(url))
            return 
        
        file_path = os.path.join(download_path, pdb_id + ".pdb")
        with open(file_path, "wb") as f:
            f.write(res.content)

In [3]:
# Just write the PDB id in order to download the pdb file

fetch_pdb("1brs")

In [4]:
# Load pdb with MDTraj

brs = md.load('1brs.pdb')

In [5]:
# Remove water molecules

brs = brs.remove_solvent()

In [6]:
# Barnase's chains

atoms_in_chain_A = brs.topology.select("chainid == 0")
atoms_in_chain_B = brs.topology.select("chainid == 1")
atoms_in_chain_C = brs.topology.select("chainid == 2")

In [7]:
# Barstar's chains

atoms_in_chain_D = brs.topology.select("chainid == 3")
atoms_in_chain_E = brs.topology.select("chainid == 4")
atoms_in_chain_F = brs.topology.select("chainid == 5")

In [8]:
# Barnase's chain's atoms

barnase_A = brs.atom_slice(atoms_in_chain_A)
barnase_B = brs.atom_slice(atoms_in_chain_B)
barnase_C = brs.atom_slice(atoms_in_chain_C)

In [9]:
# Barstar's chain's atoms

barstar_D = brs.atom_slice(atoms_in_chain_D)
barstar_E = brs.atom_slice(atoms_in_chain_E)
barstar_F = brs.atom_slice(atoms_in_chain_F)

Let's work with barnase B and barstar F since they are the most complete monomers. Given that barstar E is the bound to barnase B, barstar F needs to be fitted over E to have the complex B-F:

In [10]:
atoms_to_fit_E = []
atoms_to_fit_F = []
for atom_E in barstar_E.topology.atoms_by_name('CA'):
    for atom_F in barstar_F.topology.atoms_by_name('CA'):
        if str(atom_E)==str(atom_F):
            #print(f'{str(atom_E)} is {atom_E.index} in E and {atom_F.index} in F')
            atoms_to_fit_E.append(atom_E.index)
            atoms_to_fit_F.append(atom_F.index)

In [11]:
barstar_F_over_E = md.Trajectory.superpose(barstar_F, barstar_E, frame=0, atom_indices=atoms_to_fit_F, ref_atom_indices=atoms_to_fit_E)

In [12]:
new_barnase_barstar = barnase_B.stack(barstar_F_over_E)

In [13]:
view = nv.show_mdtraj(new_barnase_barstar)
view

NGLWidget()

In [14]:
new_barnase_barstar.save_pdb('new_barnase_barstar.pdb')

In [15]:
fixer = PDBFixer(filename='new_barnase_barstar.pdb')

In [16]:
fixer.findMissingResidues()
missing_residues = fixer.missingResidues
print(f"{len(missing_residues)} missing residues")

fixer.findNonstandardResidues()
nonstandard_residues = fixer.nonstandardResidues
print(f"{len(nonstandard_residues)} non standard residues")

fixer.findMissingAtoms()
missing_atoms = fixer.missingAtoms
missing_terminals = fixer.missingTerminals
print(f"{len(missing_atoms)} missing atoms")
print(f"{len(missing_terminals)} missing terminals")

if len(nonstandard_residues)>0:
    fixer.replaceNonstandardResidues()

if len(missing_atoms)>0:
    fixer.addMissingAtoms()

0 missing residues
0 non standard residues
6 missing atoms
0 missing terminals


In [17]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')

In [18]:
modeller = app.Modeller(fixer.topology, fixer.positions)
pH = 7.2
residues_protonated = modeller.addHydrogens(forcefield=forcefield, pH=pH)

In [19]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

In [20]:
# We need to define force groups to be able to calculate the different energy terms 

forcegroups = {}
for ii in range(system.getNumForces()):
    force = system.getForce(ii)
    print(f'{force.getName()} was in force group {force.getForceGroup()} and it is now in group {ii}')
    force.setForceGroup(ii)
    forcegroups[force] = ii

HarmonicBondForce was in force group 0 and it is now in group 0
HarmonicAngleForce was in force group 0 and it is now in group 1
NonbondedForce was in force group 0 and it is now in group 2
PeriodicTorsionForce was in force group 0 and it is now in group 3
CMMotionRemover was in force group 0 and it is now in group 4


In [24]:
# A simulation object is needed

## Integrator
step_size   = 0.002*unit.picoseconds
temperature = 0.0*unit.kelvin
friction    = 0.0/unit.picosecond # Damping para la dinámica de Langevin

integrator = mm.LangevinIntegrator(temperature, friction, step_size)

## Platform
platform_name = 'CUDA'
platform    = mm.Platform.getPlatformByName(platform_name)

In [25]:
# Creación del objeto context del sistema en vacio
context = mm.Context(system, integrator, platform)

In [26]:
# Condiciones iniciales
context.setPositions(modeller.positions)

In [27]:
# Minimizacion del sistema
state_pre_minimization = context.getState(getEnergy=True)
mm.LocalEnergyMinimizer_minimize(context)
state_post_minimization = context.getState(getEnergy=True)

In [28]:
print('Energy before minimization:', state_pre_minimization.getPotentialEnergy())
print('Energy after minimization:', state_post_minimization.getPotentialEnergy())

Energy before minimization: -13157.89453125 kJ/mol
Energy after minimization: -25002.3125 kJ/mol


In [68]:
residues_per_chain={}
atoms_per_chain={}
atoms_per_residue={}
for chain in modeller.topology.chains():
    atoms_per_chain[chain.index]=[]
    residues_per_chain[chain.index]=[]
    for residue in chain.residues():
        residues_per_chain[chain.index].append(residue.index)
        atoms_per_residue[residue.index]=[]
        for atom in residue.atoms():
            atoms_per_residue[residue.index].append(atom.index)
            atoms_per_chain[chain.index].append(atom.index)

In [30]:
energies = {}
for ff, ii in forcegroups.items():
    energies[ff.getName()] = context.getState(getEnergy=True, groups={ii}).getPotentialEnergy()

In [31]:
for ii,jj in energies.items():
    print(ii,jj)

HarmonicBondForce 410.74688720703125 kJ/mol
HarmonicAngleForce 1697.7552490234375 kJ/mol
NonbondedForce -36628.328125 kJ/mol
PeriodicTorsionForce 9517.5146484375 kJ/mol
CMMotionRemover 0.0 kJ/mol


In [32]:
energies['HarmonicBondForce']+energies['HarmonicAngleForce']+energies['NonbondedForce']+energies['PeriodicTorsionForce']

Quantity(value=-25002.31134033203, unit=kilojoule/mole)

- We should work on how to decompose de NonbondedForce: LJ contribution (VdW) and Coulomb contribution. 

In [33]:
non_bonded_force = None
for ii in range(system.getNumForces()):
    force = system.getForce(ii)
    if force.getName() == 'NonbondedForce':
        non_bonded_force = force

In [36]:
original_particle_parameters = {}
for ii in range(non_bonded_force.getNumParticles()):
    charge, sigma, epsilon = non_bonded_force.getParticleParameters(ii)
    original_particle_parameters[ii] = {'charge': charge, 'sigma': sigma, 'epsilon':epsilon}

original_exception_parameters = {}
for ii in range(non_bonded_force.getNumExceptions()):
    particle1, particle2, chargeProd, sigma, epsilon = non_bonded_force.getExceptionParameters(ii)
    original_exception_parameters[ii] = {'particle1': particle1, 'particle2': particle2, 'chargeProd': chargeProd,
                                     'sigma': sigma, 'epsilon': epsilon}

Let's compute the energy terms of the system when all charges and epsilons have been set to zero:

In [42]:
def get_non_bonded_energy(atoms_in, non_bonded_force, context):
    
    for ii, parameters in original_particle_parameters.items():
        
        charge = parameters['charge']
        sigma = parameters['sigma']
        epsilon = parameters['epsilon']
        
        if ii not in atoms_in:
            charge = 0.0 * unit.elementary_charge
            epsilon = 0.0 * unit.kilojoule_per_mole

        non_bonded_force.setParticleParameters(ii, charge, sigma, epsilon)

    for ii, parameters in original_exception_parameters.items():
        
        particle1 = parameters['particle1']
        particle2 = parameters['particle2']
        chargeProd = parameters['chargeProd']
        sigma = parameters['sigma']
        epsilon = parameters['epsilon']
        
        if (particle1 not in atoms_in) and (particle2 not in atoms_in):
            epsilon = 0.0 * unit.kilojoule_per_mole
            if chargeProd != 0.0 * unit.elementary_charge**2:
                chargeProd = 0.0000000000001 * unit.elementary_charge **2

        non_bonded_force.setExceptionParameters(ii, particle1, particle2, chargeProd,
                                                sigma, epsilon)
        
    non_bonded_force.updateParametersInContext(context)

    energy = context.getState(getEnergy=True, groups={2}).getPotentialEnergy()

    return energy

In [72]:
no_atoms = []
atoms_chain0 = atoms_per_chain[0]
atoms_chain1 = atoms_per_chain[1]
all_atoms = atoms_chain0 + atoms_chain1
    
no_atoms_energy = get_non_bonded_energy(no_atoms, non_bonded_force, context)
chain0_energy = get_non_bonded_energy(atoms_chain0, non_bonded_force, context)
chain1_energy = get_non_bonded_energy(atoms_chain1, non_bonded_force, context)
all_energy = get_non_bonded_energy(all_atoms, non_bonded_force, context)

In [73]:
print(f'Non bonded energy term with no atoms: {no_atoms_energy}')
print(f'Non bonded energy term with chain 0 atoms: {chain0_energy}')
print(f'Non bonded energy term with chain 1 atoms: {chain1_energy}')
print(f'Non bonded energy term with all atoms: {all_energy}')

Non bonded energy term with no atoms: 3.909928238954308e-07 kJ/mol
Non bonded energy term with chain 0 atoms: -19280.14453125 kJ/mol
Non bonded energy term with chain 1 atoms: -14094.1884765625 kJ/mol
Non bonded energy term with all atoms: -36628.328125 kJ/mol


In [74]:
print(f'Non bonded energy term for chain 0 vs chain 1: {all_energy-chain0_energy-chain1_energy}')

Non bonded energy term for chain 0 vs chain 1: -3253.9951171875 kJ/mol


In [75]:
energy_per_residue={}

for residue_index, atoms in tqdm(atoms_per_residue.items()):
    energy_per_residue[residue_index]=get_non_bonded_energy(atoms, non_bonded_force, context)

100%|█████████████████████████████████████████████████████████████| 199/199 [02:02<00:00,  1.63it/s]


In [76]:
energy_interfase_per_residue={}

for residue_index_0 in tqdm(residues_per_chain[0]):
    atoms_0 = atoms_per_residue[residue_index_0]
    aux_energy = get_non_bonded_energy(atoms_0+atoms_per_chain[1], non_bonded_force, context)
    energy_interfase_per_residue[residue_index_0]=aux_energy-energy_per_residue[residue_index_0]-chain1_energy

for residue_index_1 in tqdm(residues_per_chain[1]):
    atoms_1 = atoms_per_residue[residue_index_1]
    aux_energy = get_non_bonded_energy(atoms_1+atoms_per_chain[0], non_bonded_force, context)
    energy_interfase_per_residue[residue_index_1]=aux_energy-energy_per_residue[residue_index_1]-chain0_energy


100%|███████████████████████████████████████████████████████████████| 89/89 [01:32<00:00,  1.04s/it]


In [81]:
np.sum(list(energy_interfase_per_residue.values()))/2.0

Quantity(value=-3254.078502856195, unit=kilojoule/mole)

In [129]:
energy_interfase_per_residue

{0: Quantity(value=-308.1665725708008, unit=kilojoule/mole),
 1: Quantity(value=13.050323486328125, unit=kilojoule/mole),
 2: Quantity(value=-7.03173828125, unit=kilojoule/mole),
 3: Quantity(value=4.804962158203125, unit=kilojoule/mole),
 4: Quantity(value=-3.7353363037109375, unit=kilojoule/mole),
 5: Quantity(value=1.455810546875, unit=kilojoule/mole),
 6: Quantity(value=-1.6517333984375, unit=kilojoule/mole),
 7: Quantity(value=311.4040832519531, unit=kilojoule/mole),
 8: Quantity(value=-3.507171630859375, unit=kilojoule/mole),
 9: Quantity(value=-5.208099365234375, unit=kilojoule/mole),
 10: Quantity(value=-0.5933837890625, unit=kilojoule/mole),
 11: Quantity(value=281.777587890625, unit=kilojoule/mole),
 12: Quantity(value=-8.21722412109375, unit=kilojoule/mole),
 13: Quantity(value=-6.22637939453125, unit=kilojoule/mole),
 14: Quantity(value=-9.625396728515625, unit=kilojoule/mole),
 15: Quantity(value=-7.277679443359375, unit=kilojoule/mole),
 16: Quantity(value=-3.076751708984

In [132]:
aux = [ii._value for ii in energy_interfase_per_residue.values()]
aux = np.array(aux)
max_abs_val = max(abs(aux.min()), abs(aux.max()))

from nglview.color import _ColorScheme
from matplotlib.colors import Normalize, to_hex
from matplotlib import cm
cmap = cm.get_cmap('RdYlBu_r')
vmin = -max_abs_val
vmax = max_abs_val

norm = Normalize(vmin=vmin,vmax=vmax)
scheme = _ColorScheme([[to_hex(cmap(norm(ii))), str(jj)] for ii,jj in zip(aux, range(len(aux)))], label='user')

In [133]:
scheme.data

{'data': [['#cdeaf3', '0'],
  ['#fffdbc', '1'],
  ['#feffc0', '2'],
  ['#fffebe', '3'],
  ['#feffc0', '4'],
  ['#fffebe', '5'],
  ['#feffc0', '6'],
  ['#fecc7e', '7'],
  ['#feffc0', '8'],
  ['#feffc0', '9'],
  ['#feffc0', '10'],
  ['#fed485', '11'],
  ['#feffc0', '12'],
  ['#feffc0', '13'],
  ['#fdfec2', '14'],
  ['#feffc0', '15'],
  ['#feffc0', '16'],
  ['#fffebe', '17'],
  ['#cfebf3', '18'],
  ['#fdfec2', '19'],
  ['#fffdbc', '20'],
  ['#fece7f', '21'],
  ['#feffc0', '22'],
  ['#fcfec5', '23'],
  ['#fffebe', '24'],
  ['#fdfec2', '25'],
  ['#394fa1', '26'],
  ['#fcfec5', '27'],
  ['#fdb366', '28'],
  ['#fcfec5', '29'],
  ['#fbfdc7', '30'],
  ['#fdfec2', '31'],
  ['#fdfec2', '32'],
  ['#feffc0', '33'],
  ['#feffc0', '34'],
  ['#fbfdc7', '35'],
  ['#fafdc9', '36'],
  ['#f3fbd4', '37'],
  ['#b0dcea', '38'],
  ['#fdfec2', '39'],
  ['#fffdbc', '40'],
  ['#fdfec2', '41'],
  ['#feffc0', '42'],
  ['#fdc576', '43'],
  ['#feffc0', '44'],
  ['#fffebe', '45'],
  ['#fffebe', '46'],
  ['#feffc0', '

In [134]:
view = nv.show_mdtraj(new_barnase_barstar)
view.clear()
view.add_cartoon(selection='all', color=scheme)
view

NGLWidget()

In [62]:
crossed_energies={}
for residue_index_0 in tqdm(residues_per_chain[0]):
    crossed_energies[residue_index_0]={}
    atoms_0 = atoms_per_residue[residue_index_0]
    for residue_index_1 in residues_per_chain[1]:
        atoms_1 = atoms_per_residue[residue_index_1]
        atoms_pair = atoms_0+atoms_1
        aux_energy = get_non_bonded_energy(atoms_pair, non_bonded_force, context)
        crossed_energies[residue_index_0][residue_index_1]=aux_energy-energy_per_residue[residue_index_0]-energy_per_residue[residue_index_1]

  5%|██▊                                                          | 5/110 [04:34<1:36:06, 54.92s/it]


KeyboardInterrupt: 

{110: Quantity(value=90.97820281982422, unit=kilojoule/mole),
 111: Quantity(value=34.23235321044922, unit=kilojoule/mole),
 112: Quantity(value=0.08483123779296875, unit=kilojoule/mole),
 113: Quantity(value=-0.41168975830078125, unit=kilojoule/mole),
 114: Quantity(value=0.12279510498046875, unit=kilojoule/mole),
 115: Quantity(value=-0.5396194458007812, unit=kilojoule/mole),
 116: Quantity(value=0.03258514404296875, unit=kilojoule/mole),
 117: Quantity(value=-28.64759063720703, unit=kilojoule/mole),
 118: Quantity(value=-0.16040802001953125, unit=kilojoule/mole),
 119: Quantity(value=0.49202728271484375, unit=kilojoule/mole),
 120: Quantity(value=28.841697692871094, unit=kilojoule/mole),
 121: Quantity(value=-0.29206085205078125, unit=kilojoule/mole),
 122: Quantity(value=-0.42447662353515625, unit=kilojoule/mole),
 123: Quantity(value=-0.7099075317382812, unit=kilojoule/mole),
 124: Quantity(value=-32.90668487548828, unit=kilojoule/mole),
 125: Quantity(value=-0.32880401611328125, 

# Of interest

https://github.com/openmm/openmm/issues/1463    
https://github.com/openmm/openmm/issues/387    
https://github.com/openmm/openmm/issues/2682    
https://github.com/openmm/openmm/issues/1830    
https://openmm.github.io/openmm-cookbook/dev/notebooks/cookbook/Analyzing%20Energy%20Contributions.html    
https://openmm.github.io/openmm-cookbook/dev/notebooks/cookbook/Querying%20Charges%20and%20Other%20Parameters.html    


https://github.com/openmm/openmm/issues/3178    
https://github.com/openmm/openmm/issues/3169    
https://github.com/openmm/openmm/issues/2880    
https://github.com/openmm/openmm/issues/2835    
https://github.com/openmm/openmm/issues/2682    

https://chemrxiv.org/engage/api-gateway/chemrxiv/assets/orp/resource/item/60c758a8567dfe4abcec68b3/original/sbm-open-mm-a-builder-of-structure-based-models-for-open-mm.pdf